# Logistic Regression Model 

In [1]:
#read data from csv file
import pandas as pd

crash_df = pd.read_csv("C:/Users/willi/Desktop/MLA_Crash_Paper/Crash_Data.csv")
crash_df.head()

,X,Y,OBJECTID,DOCUMENT_NBR,CRASH_YEAR,CRASH_DT,CRASH_MILITARY_TM,CRASH_SEVERITY,K_PEOPLE,A_PEOPLE,...,AREA_TYPE,SYSTEM,VSP,OWNERSHIP,PLAN_DISTRICT,MPO_NAME,RTE_NM,RNS_MP,NODE,OFFSET
0,-78.870556,38.645544,1,152915206,2015,2015/10/16 17:00:00+00,1700,A,0,2,...,Rural,VDOT Primary,2,1. State Hwy Agency,Central Shenandoah,NaN,R-VA SR00259SB,9.63,586049.0,834.0
1,-77.134973,38.943531,2,153060033,2015,2015/10/10 17:00:00+00,2350,O,0,0,...,Urban,NonVDOT secondary,7,4. Federal Roads,Northern Virginia,NOVA,R-VA OT90072NB,0.41,NaN,NaN
2,-79.499399,36.660702,3,153035144,2015,2015/10/29 17:00:00+00,1124,B,0,0,...,Rural,VDOT Secondary,6,1. State Hwy Agency,West Piedmont,NaN,R-VA071SC00750SB,5.90,518624.0,137.0
3,-77.336244,38.545770,4,150975078,2015,2015/04/06 17:00:00+00,900,O,0,0,...,Urban,VDOT Secondary,7,1. State Hwy Agency,Northern Virginia,NOVA,R-VA076SC00619EB,25.74,NaN,NaN
4,-79.337225,36.581073,5,152335159,2015,2015/08/21 17:00:00+00,751,O,0,0,...,Urban,NonVDOT primary,6,3. City or Town Hwy Agency,West Piedmont,DAN,R-VA US00058WB,301.27,519895.0,876.0


In [2]:
#create target variable as fatal or severe injury

crash_df['CRASH_SEVERITY'] = crash_df["CRASH_SEVERITY"].apply(lambda x: 1 if (x =='A') | (x=='K') else 0)
crash_df["CRASH_SEVERITY"].value_counts()

CRASH_SEVERITY
0    922559
1     54787
Name: count, dtype: int64

In [3]:
#create new dataset which includes variables that we are interested in\

crash_df = crash_df[['CRASH_SEVERITY', 'WEATHER_CONDITION', 'LIGHT_CONDITION', 'ALCOHOL_NOTALCOHOL', 'DISTRACTED_NOTDISTRACTED', 'DRUG_NODRUG', 'SPEED_NOTSPEED', 'SPEED_DIFF_MAX']]
crash_df.isnull().sum()

CRASH_SEVERITY                   0
WEATHER_CONDITION                0
LIGHT_CONDITION                  0
ALCOHOL_NOTALCOHOL               0
DISTRACTED_NOTDISTRACTED         0
DRUG_NODRUG                      0
SPEED_NOTSPEED                   0
SPEED_DIFF_MAX              783400
dtype: int64

In [4]:
#imputing missing data for column SPEED_DIFF_MAX to be 0
import numpy as np
crash_df[crash_df['SPEED_DIFF_MAX'].isnull()]['SPEED_NOTSPEED'].value_counts()  #proves that when SPEED_DIFF_MAX is NaN the driver did not go over the speed limit
crash_df['SPEED_DIFF_MAX'] = crash_df['SPEED_DIFF_MAX'].replace(np.nan, 0)
crash_df.isnull().sum()

CRASH_SEVERITY              0
WEATHER_CONDITION           0
LIGHT_CONDITION             0
ALCOHOL_NOTALCOHOL          0
DISTRACTED_NOTDISTRACTED    0
DRUG_NODRUG                 0
SPEED_NOTSPEED              0
SPEED_DIFF_MAX              0
dtype: int64

In [5]:
for i in ['WEATHER_CONDITION', 'LIGHT_CONDITION', 'ALCOHOL_NOTALCOHOL', 'DISTRACTED_NOTDISTRACTED', 'DRUG_NODRUG']:
    print(crash_df.groupby([i]).CRASH_SEVERITY.sum())

WEATHER_CONDITION
1. No Adverse Condition (Clear/Cloudy)    47365
10. Blowing Sand, Soil, Dirt, or Snow        19
11. Severe Crosswinds                        33
3. Fog                                      322
4. Mist                                     796
5. Rain                                    5543
6. Snow                                     477
7. Sleet/Hail                               163
8. Smoke/Dust                                 4
9. Other                                     65
Not Applicable                                0
Name: CRASH_SEVERITY, dtype: int64
LIGHT_CONDITION
1. Dawn                                 1160
2. Daylight                            32492
3. Dusk                                 1630
4. Darkness - Road Lighted              7734
5. Darkness - Road Not Lighted         11612
6. Darkness - Unknown Road Lighting      124
7. Unknown                                35
Name: CRASH_SEVERITY, dtype: int64
ALCOHOL_NOTALCOHOL
No     45656
Yes     9131
Name: CR

In [6]:
crash_df['WEATHER'] = crash_df["WEATHER_CONDITION"].apply(lambda x: "Bad_Weather" if x in ("10. Blowing Sand, Soil, Dirt, or Snow", "11. Severe Crosswinds", "3. Fog", "4. Mist", "6. Snow", "7. Sleet/Hail", "8. Smoke/Dust", "9. Other", "Not Applicable") else x)
crash_df.WEATHER.value_counts()

WEATHER
1. No Adverse Condition (Clear/Cloudy)    808585
5. Rain                                   128227
Bad_Weather                                40534
Name: count, dtype: int64

In [7]:
crash_df['LIGHT'] = crash_df["LIGHT_CONDITION"].apply(lambda x: "Semi_Dark" if x in ("1. Dawn", "3. Dusk", "4. Darkness - Road Lighted", "6. Darkness - Unknown Road Lighting", "7. Unknown") else x)
crash_df.LIGHT.value_counts()

LIGHT
2. Daylight                       644092
Semi_Dark                         187934
5. Darkness - Road Not Lighted    145320
Name: count, dtype: int64

In [8]:
crash_df = crash_df.drop(columns=['WEATHER_CONDITION', 'LIGHT_CONDITION'])
crash_df.sample(10)

,CRASH_SEVERITY,ALCOHOL_NOTALCOHOL,DISTRACTED_NOTDISTRACTED,DRUG_NODRUG,SPEED_NOTSPEED,SPEED_DIFF_MAX,WEATHER,LIGHT
843621,0,No,Yes,No,No,0.0,5. Rain,5. Darkness - Road Not Lighted
938013,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
509504,0,No,Yes,No,No,0.0,5. Rain,2. Daylight
149486,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
568961,0,No,No,No,Yes,5.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
667458,1,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
368740,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
602880,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
148992,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight
502193,0,No,No,No,No,0.0,1. No Adverse Condition (Clear/Cloudy),2. Daylight


In [9]:
crash_df = pd.get_dummies(crash_df, dtype=int)
crash_df.head()

,CRASH_SEVERITY,SPEED_DIFF_MAX,ALCOHOL_NOTALCOHOL_No,ALCOHOL_NOTALCOHOL_Yes,DISTRACTED_NOTDISTRACTED_No,DISTRACTED_NOTDISTRACTED_Yes,DRUG_NODRUG_No,DRUG_NODRUG_Yes,SPEED_NOTSPEED_No,SPEED_NOTSPEED_Yes,WEATHER_1. No Adverse Condition (Clear/Cloudy),WEATHER_5. Rain,WEATHER_Bad_Weather,LIGHT_2. Daylight,LIGHT_5. Darkness - Road Not Lighted,LIGHT_Semi_Dark
0,1,3.0,1,0,0,1,1,0,0,1,1,0,0,1,0,0
1,0,10.0,0,1,0,1,1,0,0,1,1,0,0,0,1,0
2,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
3,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
4,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0


In [10]:
crash_df = crash_df.replace({'5. Darkness - Road Not Lighted': 'Dark', '2. Daylight': "Daylight", "1. No Adverse Condition (Clear/Cloudy)": "Clear", "5. Rain": "Rain"})

In [11]:
# save the data as a parquet file
crash_df.to_csv('C:/Users/willi/Desktop/MLA_Crash_Paper/All_Model.csv', sep=',')

In [12]:
crash_df.head()

,CRASH_SEVERITY,SPEED_DIFF_MAX,ALCOHOL_NOTALCOHOL_No,ALCOHOL_NOTALCOHOL_Yes,DISTRACTED_NOTDISTRACTED_No,DISTRACTED_NOTDISTRACTED_Yes,DRUG_NODRUG_No,DRUG_NODRUG_Yes,SPEED_NOTSPEED_No,SPEED_NOTSPEED_Yes,WEATHER_1. No Adverse Condition (Clear/Cloudy),WEATHER_5. Rain,WEATHER_Bad_Weather,LIGHT_2. Daylight,LIGHT_5. Darkness - Road Not Lighted,LIGHT_Semi_Dark
0,1,3.0,1,0,0,1,1,0,0,1,1,0,0,1,0,0
1,0,10.0,0,1,0,1,1,0,0,1,1,0,0,0,1,0
2,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
3,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0
4,0,0.0,1,0,1,0,1,0,1,0,1,0,0,1,0,0


In [13]:
#standardize variable and create dummy variable
import math

mean_dff = crash_df[(crash_df['SPEED_DIFF_MAX']<100)]['SPEED_DIFF_MAX'].mean() # max_diff will be capped at 100
std_dff = crash_df[(crash_df['SPEED_DIFF_MAX']<100)]['SPEED_DIFF_MAX'].std()

crash_df['S_SPEED_DIFF'] = crash_df["SPEED_DIFF_MAX"].apply(lambda x: (x-mean_dff)/std_dff if x<=100 else (100-mean_dff)/std_dff)

In [14]:
crash_df = crash_df.drop(columns=['SPEED_DIFF_MAX', 'SPEED_NOTSPEED_No', 'SPEED_NOTSPEED_Yes'])
crash_df.sample(10)

,CRASH_SEVERITY,ALCOHOL_NOTALCOHOL_No,ALCOHOL_NOTALCOHOL_Yes,DISTRACTED_NOTDISTRACTED_No,DISTRACTED_NOTDISTRACTED_Yes,DRUG_NODRUG_No,DRUG_NODRUG_Yes,WEATHER_1. No Adverse Condition (Clear/Cloudy),WEATHER_5. Rain,WEATHER_Bad_Weather,LIGHT_2. Daylight,LIGHT_5. Darkness - Road Not Lighted,LIGHT_Semi_Dark,S_SPEED_DIFF
735630,0,1,0,1,0,1,0,1,0,0,1,0,0,-0.374274
572555,0,1,0,1,0,1,0,1,0,0,0,1,0,-0.374274
860089,0,0,1,1,0,1,0,1,0,0,0,0,1,-0.374274
305164,0,1,0,1,0,1,0,1,0,0,0,0,1,-0.374274
946666,0,1,0,1,0,1,0,1,0,0,1,0,0,-0.374274
297619,0,0,1,0,1,1,0,1,0,0,0,0,1,-0.374274
591189,0,1,0,1,0,1,0,1,0,0,1,0,0,-0.374274
698071,1,1,0,1,0,1,0,1,0,0,1,0,0,1.697502
612698,0,1,0,1,0,1,0,0,1,0,1,0,0,-0.374274
447433,0,1,0,1,0,1,0,1,0,0,0,0,1,-0.374274


In [15]:
# describe distribution of standardized and transformed speed diff
crash_df['S_SPEED_DIFF'].describe()

count    977346.000000
mean          0.000316
std           1.002111
min          -0.374274
25%          -0.374274
50%          -0.374274
75%          -0.374274
max          13.437562
Name: S_SPEED_DIFF, dtype: float64

In [16]:
#split sample to build logistic regression and validate model
from sklearn.model_selection import train_test_split
Y = crash_df['CRASH_SEVERITY']
X = crash_df.drop('CRASH_SEVERITY', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, stratify=Y)

In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [19]:
y_pred = model.predict(x_test)
y_pred_proba_LR = model.predict_proba(x_test)[:, 1]
y_pred_P_LR = pd.DataFrame(y_pred_proba_LR, columns=['LR_Proba'])
y_pred_P_LR.head()

,LR_Proba
0,0.045245
1,0.137891
2,0.039152
3,0.058722
4,0.056716


In [20]:
y_pred_P_LR.to_csv('C:/Users/willi/Desktop/MLA_Crash_Paper/LR_Proba.csv')

In [83]:
# understand model
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97    184513
           1       0.56      0.00      0.01     10957

    accuracy                           0.94    195470
   macro avg       0.75      0.50      0.49    195470
weighted avg       0.92      0.94      0.92    195470



In [84]:
from sklearn.metrics import roc_auc_score
# Step 2: Make predictions using the XGBoost model
y_pred_proba = model.predict_proba(x_test)[:, 1]  # Use probabilities for Somers' D calculation
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.6194819877882709


In [85]:
#Calculating Somers' D value
sd = 2*auc-1
print(sd)

0.2389639755765418


In [21]:
# print out the coefficient of logistic regression model
def logreg_to_dict(clf: LogisticRegression, feature_names: list[str]) -> dict[str, float]:
    coefs = np.concatenate([clf.intercept_, clf.coef_.squeeze()])
    return dict(zip(["intercept"] + feature_names, coefs))
coeff = logreg_to_dict(model, x_train.columns.to_list())
coeff

{'intercept': -0.6900736613927657,
 'ALCOHOL_NOTALCOHOL_No': -0.9054872194560183,
 'ALCOHOL_NOTALCOHOL_Yes': 0.21673333152437796,
 'DISTRACTED_NOTDISTRACTED_No': -0.35215613616411673,
 'DISTRACTED_NOTDISTRACTED_Yes': -0.33659775176751866,
 'DRUG_NODRUG_No': -0.6910656053638855,
 'DRUG_NODRUG_Yes': 0.0023117174322491068,
 'WEATHER_1. No Adverse Condition (Clear/Cloudy)': -0.03074984979175695,
 'WEATHER_5. Rain': -0.33695928987097556,
 'WEATHER_Bad_Weather': -0.32104474826890156,
 'LIGHT_2. Daylight': -0.29572118892871785,
 'LIGHT_5. Darkness - Road Not Lighted': -0.02077802097900389,
 'LIGHT_Semi_Dark': -0.37225467802392265,
 'S_SPEED_DIFF': 0.22473239239474593}